In [0]:
!rm -r /root/data
!rm -r video.avi
!rm -r label.txt
!mkdir -p /root/data
!mkdir -p /root/data/samples_cut
from google.colab import files
import cv2
uploaded = files.upload()
cap = cv2.VideoCapture('video.avi')
ret,frame = cap.read()
name = 0
while ret:
    cv2.imwrite('/root/data/' + str(name) + '.jpg',frame[120:520,40:440])
    name += 1
    ret,frame = cap.read()
!mv label.txt /root/data/
!ls /root/data/

以上負責處理訓練用檔案跟虛擬機的基本建置


---



In [0]:
#引用部分
import numpy as np
import cv2
from keras.preprocessing import image
from keras.utils import to_categorical
import json
import io

KERAS_MODEL_NAME = 'dnn_model.model'
KERAS_MODEL_WEIG = 'dnn_model.h5'

#預留必要的變數名稱
X_Train = []
Y_Train = []
Y_Train_temp = []

#讀取標籤
with open('/root/data/label.txt') as f:
    Y_Train_temp = json.load(f)

#讀取照片
for i in range(len(Y_Train_temp)):
    img = cv2.imread('/root/data/' + str(i) + '.jpg')
    img_array = image.img_to_array(cv2.resize(img,(160,160),interpolation=cv2.INTER_CUBIC))
    Y_Train.append(Y_Train_temp[i])
    X_Train.append(img_array.reshape((160,160,3))/255)
    
#轉換成tensorflow能吃的格式
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)
Y_Train = to_categorical(Y_Train)

#設計網路開始
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

model = Sequential()
#建立卷積層
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',
                 input_shape=(160,160,3),activation='relu'))
model.add(Conv2D(filters=48,kernel_size=(1,1),padding='same',
                 input_shape=(160,160,3),activation='relu'))
#建立池化層
model.add(MaxPooling2D(pool_size=(2,2)))

#建立卷積層
model.add(Conv2D(filters=24,kernel_size=(3,3),padding='same',
                 input_shape=(160,160,3),activation='relu'))
model.add(Conv2D(filters=24,kernel_size=(3,3),padding='same',
                 input_shape=(160,160,3),activation='relu'))

#建立池化層
model.add(MaxPooling2D(pool_size=(2,2)))

#建立全連結層
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

#建立輸出層
model.add(Dense(4, activation='softmax'))

#設定訓練參數
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#印出來看看
model.summary()

#訓練
train_history = model.fit(x=X_Train,y=Y_Train, validation_split=0.1,
                          epochs=20, batch_size=50, verbose=1,shuffle=True)

print("[Info] Serialized Keras model to %s..." % (KERAS_MODEL_NAME))

#儲存模型
with io.open(KERAS_MODEL_NAME, 'w', encoding = 'utf-8') as f:
    f.write(model.to_json()) 
model.save_weights(KERAS_MODEL_WEIG)


以上為訓練。GPU並不需要特別重寫程式碼，但TPU就要了，甚至有時候需要調網路架構


---



In [0]:
#下載訓練好的模型
from google.colab import files
files.download('dnn_model.model')
files.download('dnn_model.h5')

將模型載下來，注意虛擬機其實會先把模型上傳到某個雲端硬碟再下載，所以需要等他傳一下子